In [1]:
import os
import sys
import json
import keras
import datetime
import numpy as np
import skimage.draw
import cv2
from mrcnn.visualize import display_instances
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
# Root directory of the project
ROOT_DIR = "D:\\UTM\\SEM 8\\FYP 2\\Rockmelon"
#LOG_ROOT_DIR = "C:\\Users\\ainul\\Documents\\Artificial Intelligent\\Rockmelon\\logs\\object20220621T1329"

In [3]:
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils

In [4]:
# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Test for resume training
#COCO_WEIGHTS_PATH = os.path.join(LOG_ROOT_DIR, "mask_rcnn_object_0050.h5")

In [5]:
# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

In [6]:
class CustomConfig(Config):
    """Configuration for training on the custom  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "object"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 2  # Background + Downey Mildew, Powdery Mildew

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 300

    # Skip detections with < 80% confidence
    DETECTION_MIN_CONFIDENCE = 0.9


#  Dataset

In [7]:
class CustomDataset(utils.Dataset):

	def load_custom(self, dataset_dir, subset):
		"""Load a subset of the Dog-Cat dataset.
		dataset_dir: Root directory of the dataset.
		subset: Subset to load: train or test_val
		"""
		# Add classes. We have only one class to add.
		self.add_class("object", 1, "Downey_Mildew")
		self.add_class("object", 2, "Powdery_Mildew")

		# Train or validation dataset?
		assert subset in ["train", "test_val"]
		dataset_dir = os.path.join(dataset_dir, subset)

		# Load annotations
		# VGG Image Annotator saves each image in the form:
		# { 'filename': '28503151_5b5b7ec140_b.jpg',
		#   'regions': {
		#       '0': {
		#           'region_attributes': {},
		#           'shape_attributes': {
		#               'all_points_x': [...],
		#               'all_points_y': [...],
		#               'name': 'polygon'}},
		#       ... more regions ...
		#   },
		#   'size': 100202
		# }
		# We mostly care about the x and y coordinates of each region
		annotations1 = json.load(open(os.path.join(dataset_dir, 'demo_json.json')))
		#annotations1 = json.load(open('C:\\Users\\ainul\\Documents\\Artificial Intelligent\\AarohiMRCNN\\laptopPhoneTablet\\dataset\\train\\demo_json.json'))
		
		# print(annotations1)
		annotations = list(annotations1.values())  # don't need the dict keys

		# The VIA tool saves images in the JSON even if they don't have any
		# annotations. Skip unannotated images.
		annotations = [a for a in annotations if a['regions']]
		
		# Add images
		for a in annotations:
			# print(a)
			# Get the x, y coordinates of points of the polygons that make up
			# the outline of each object instance. There are stores in the
			# shape_attributes (see json format above)
			polygons = [r['shape_attributes'] for r in a['regions']] 
			objects = [s['region_attributes']['names'] for s in a['regions']]
			print("objects:",objects)
			name_dict = {"Downey_Mildew": 1,"Powdery_Mildew": 2}
			#name_dict = {"Horse": 1,"Man": 2} #,"xyz": 3}
			# key = tuple(name_dict)
			num_ids = [name_dict[a] for a in objects]
	 
			# num_ids = [int(n['Event']) for n in objects]
			# load_mask() needs the image size to convert polygons to masks.
			# Unfortunately, VIA doesn't include it in JSON, so we must read
			# the image. This is only manageable since the dataset is tiny.
			print("numids",num_ids)
			image_path = os.path.join(dataset_dir, a['filename'])
			image = skimage.io.imread(image_path)
			height, width = image.shape[:2]

			self.add_image(
				"object",  ## for a single class just add the name here
				image_id=a['filename'],  # use file name as a unique image id
				path=image_path,
				width=width, height=height,
				polygons=polygons,
				num_ids=num_ids
				)

	def load_mask(self, image_id):
		"""Generate instance masks for an image.
	   Returns:
		masks: A bool array of shape [height, width, instance count] with
			one mask per instance.
		class_ids: a 1D array of class IDs of the instance masks.
		"""
		# If not a Dog-Cat dataset image, delegate to parent class.
		image_info = self.image_info[image_id]
		if image_info["source"] != "object":
			return super(self.__class__, self).load_mask(image_id)

		# Convert polygons to a bitmap mask of shape
		# [height, width, instance_count]
		info = self.image_info[image_id]
		if info["source"] != "object":
			return super(self.__class__, self).load_mask(image_id)
		num_ids = info['num_ids']
		mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
						dtype=np.uint8)
		for i, p in enumerate(info["polygons"]):
			# Get indexes of pixels inside the polygon and set them to 1
			rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

			mask[rr, cc, i] = 1

		# Return mask, and array of class IDs of each instance. Since we have
		# one class ID only, we return an array of 1s
		# Map class names to class IDs.
		num_ids = np.array(num_ids, dtype=np.int32)
		return mask, num_ids #np.ones([mask.shape[-1]], dtype=np.int32)

	def image_reference(self, image_id):
		"""Return the path of the image."""
		info = self.image_info[image_id]
		if info["source"] == "object":
			return info["path"]
		else:
			super(self.__class__, self).image_reference(image_id)

## Train and Validation

In [8]:
def train(model):
    """Train the model."""
    # Training dataset.
    dataset_train = CustomDataset()
    dataset_train.load_custom("D:\\UTM\\SEM 8\\FYP 2\\Rockmelon\\datasets_real", "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = CustomDataset()
    dataset_val.load_custom("D:\\UTM\\SEM 8\\FYP 2\\Rockmelon\\datasets_real", "test_val")
    dataset_val.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=40,
                layers='heads')

    # Training - Stage 2
    #Finetune layers from ResNet stage 4 and up
    print("Fine tune Resnet stage 4 and up")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=25,
                layers='4+')

    # Training - Stage 3
    # Fine tune all layers
    print("Fine tune all layers")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE / 10,
                epochs=10,
                layers='all')

In [9]:
config = CustomConfig()
config.display()
model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=DEFAULT_LOGS_DIR)


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                15
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.0001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE        

In [10]:
weights_path = COCO_WEIGHTS_PATH
        # Download weights file
if not os.path.exists(weights_path):
  utils.download_trained_weights(weights_path)

In [11]:
model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])

In [12]:
train(model)

objects: ['Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew']
numids [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
objects: ['Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey_Mildew', 'Downey

c:\Users\ainul\anaconda3\envs\tf2-gpu-api\lib\site-packages\tensorflow\python\framework\indexed_slices.py:435: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("training/SGD/gradients/gradients/roi_align_classifier/concat_grad/sub:0", shape=(None,), dtype=int32), values=Tensor("training/SGD/gradients/gradients/roi_align_classifier/concat_grad/GatherV2_2:0", shape=(None, 7, 7, 256), dtype=float32), dense_shape=Tensor("training/SGD/gradients/gradients/roi_align_classifier/concat_grad/Shape:0", shape=(4,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
c:\Users\ainul\anaconda3\envs\tf2-gpu-api\lib\site-packages\tensorflow\python\framework\indexed_slices.py:435: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("training/SGD/gradients/gradients/roi_align_classifier/concat_grad/sub_1:0", shape=(None,), dtype=int32), values=Tensor("training/SGD/gradients/gradients/roi_align_classi

300/300 [==============================] - ETA: 0s - batch: 149.5000 - size: 2.0000 - loss: 3.0821 - rpn_class_loss: 0.5709 - rpn_bbox_loss: 1.3209 - mrcnn_class_loss: 0.1607 - mrcnn_bbox_loss: 0.5137 - mrcnn_mask_loss: 0.5159

c:\Users\ainul\anaconda3\envs\tf2-gpu-api\lib\site-packages\tensorflow\python\keras\engine\training.py:2323: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


300/300 [==============================] - 2389s 8s/step - batch: 149.5000 - size: 2.0000 - loss: 3.0821 - rpn_class_loss: 0.5709 - rpn_bbox_loss: 1.3209 - mrcnn_class_loss: 0.1607 - mrcnn_bbox_loss: 0.5137 - mrcnn_mask_loss: 0.5159 - val_loss: 2.0676 - val_rpn_class_loss: 0.3233 - val_rpn_bbox_loss: 0.7072 - val_mrcnn_class_loss: 0.2447 - val_mrcnn_bbox_loss: 0.3817 - val_mrcnn_mask_loss: 0.4106
Epoch 2/40
300/300 [==============================] - 2308s 8s/step - batch: 149.5000 - size: 2.0000 - loss: 2.4295 - rpn_class_loss: 0.3294 - rpn_bbox_loss: 1.0992 - mrcnn_class_loss: 0.1753 - mrcnn_bbox_loss: 0.3942 - mrcnn_mask_loss: 0.4315 - val_loss: 1.9146 - val_rpn_class_loss: 0.2852 - val_rpn_bbox_loss: 0.6134 - val_mrcnn_class_loss: 0.2855 - val_mrcnn_bbox_loss: 0.3559 - val_mrcnn_mask_loss: 0.3746
Epoch 3/40
300/300 [==============================] - 2314s 8s/step - batch: 149.5000 - size: 2.0000 - loss: 2.3717 - rpn_class_loss: 0.3062 - rpn_bbox_loss: 1.0946 - mrcnn_class_loss: 0.19